In [ ]:
from pathlib import Path
import pandas as pd
import xmltodict
from tqdm.notebook import tqdm
from zeep import Client

In [ ]:
path = Path("data").resolve()

path.mkdir(parents=True, exist_ok=True)
path.exists(), pat

In [ ]:

wsdl_url = 'API USADA'

client = Client(wsdl_url)
dt_inicio = '2024-06-01'  # Data de início no formato AAAA-MM-DD
dt_final = ''  # Data final no formato AAAA-MM-DD

In [ ]:
list_ufs = ['AC', 'AL', 'AM', 'AP', 'BA', 'CE', 'DF', 'ES', 'GO', 'MA', 'MG', 'MS', 'MT', 'PA', 'PB', 'PE', 'PI', 'PR',
            'RJ', 'RN', 'RO', 'RR', 'RS', 'SC', 'SE', 'SP', 'TO']

ind_uf = list_ufs[0]

df_servico = pd.DataFrame()
ind_uf

In [ ]:
response_servico = client.service.servico(dt_inicio, dt_final, ind_uf)

path.joinpath(f"servico_{ind_uf}.xml").write_text(response_servico, encoding='utf-8')

response_servico[:1000]

In [ ]:
dic = xmltodict.parse(response_servico)
dic['RESULTADO']['ROW'][0]

In [ ]:
def get_data(_ind_uf: str, _dt_inicio: str, _dt_final: str):
    print('uf:', _ind_uf)
    
    _response_servico = client.service.servico(_dt_inicio, _dt_final, _ind_uf)
    _response_servico = _response_servico.replace('&', '&amp;').replace('<a href=', '&lt;a href=')
    
    print('len:', len(_response_servico))
    
    _filepath = path.joinpath(f"servico_{_ind_uf}.xml")
    _filepath.open('w+', encoding='utf-8').write(_response_servico)
    
    print('filepath:', _filepath)
    
    try:
        _dic = xmltodict.parse(_response_servico, encoding='utf-8')
        
        _result = _dic.get('RESULTADO', {})
        _row = _result.get('ROW', [])
        
        print('dic:', _dic.keys())
        print('result:', _result.keys())
        print('len_row:', len(_row))
    except Exception as e:
        print('ERROR:', uf, _filepath, e)
        
        return _filepath, e
    
    return _row


rows = get_data('AC', dt_inicio, dt_final)
print(len(rows))

In [ ]:
uf_rows = []
uf_errors = {}

for uf in tqdm(list_ufs):
    rows = get_data(uf, dt_inicio, dt_final)
    
    if isinstance(rows, tuple):
        uf_errors[uf] = rows
        
        continue
    
    uf_rows.extend(rows)
    print(len(rows))

In [ ]:
print(*[f"{k}: {f!s} - {e!s}" for k, (f, e) in uf_errors.items()], sep='\n')

In [ ]:
path.joinpath('.csv').write_text('\n'.join([str(row) for row in uf_rows]), encoding='utf-8')

In [ ]:
uf_row = uf_rows[0]

uf_row

In [ ]:
uf_row['CPF_SUBSTITUTO'] is None

In [ ]:
import pandas as pd

df = pd.DataFrame(uf_rows)

df

In [ ]:
df[['DT_INATIVACAO', 'DT_INSTALACAO', 'DAT_INCLUSAO', 'DAT_ALTERACAO', 'DAT_ALTERACAO_SUBSTITUTO']]

In [ ]:
pd.to_datetime(df[df['DT_INATIVACAO'].notnull()]['DT_INATIVACAO'])

In [ ]:
df_transformed = df.copy()

df_transformed['DT_INATIVACAO'] = pd.to_datetime(df_transformed['DT_INATIVACAO'], format='%Y-%m-%d', errors='coerce')
df_transformed['DT_INSTALACAO'] = pd.to_datetime(df_transformed['DT_INSTALACAO'], format='%Y-%m-%d', errors='coerce')
df_transformed['DAT_INCLUSAO'] = pd.to_datetime(df_transformed['DAT_INCLUSAO'], format='%Y-%m-%d', errors='coerce')
df_transformed['DAT_ALTERACAO'] = pd.to_datetime(df_transformed['DAT_ALTERACAO'], format='%Y-%m-%d', errors='coerce')
df_transformed['DAT_ALTERACAO_SUBSTITUTO'] = pd.to_datetime(df_transformed['DAT_ALTERACAO_SUBSTITUTO'],
                                                            format='%Y-%m-%d', errors='coerce')
df_transformed['DAT_INCLUSAO_SUBSTITUTO'] = pd.to_datetime(df_transformed['DAT_INCLUSAO_SUBSTITUTO'], format='%Y-%m-%d',
                                                           errors='coerce')
df_transformed['DT_INGRESSO_TITULAR'] = pd.to_datetime(df_transformed['DT_INGRESSO_TITULAR'], format='%Y-%m-%d',
                                                       errors='coerce')
df_transformed['DT_NOMEACAO_TITULAR'] = pd.to_datetime(df_transformed['DT_NOMEACAO_TITULAR'], format='%Y-%m-%d',
                                                       errors='coerce')
df_transformed['DT_ASSUNCAO_SERVENTIA_TITULAR'] = pd.to_datetime(df_transformed['DT_ASSUNCAO_SERVENTIA_TITULAR'],
                                                                 format='%Y-%m-%d', errors='coerce')
df_transformed['DT_COLACAO_GRAU_TITULAR'] = pd.to_datetime(df_transformed['DT_COLACAO_GRAU_TITULAR'], format='%Y-%m-%d',
                                                           errors='coerce')
df_transformed['SEQ'] = df_transformed['SEQ'].astype(int)
df_transformed['COD_IBGE'] = df_transformed['COD_IBGE'].astype('Int32')
df_transformed['BACHARELADO_TITULAR'] = df_transformed['BACHARELADO_TITULAR'].astype(bool)
df_transformed['FORMA_INGRESSO_TITULAR'] = df_transformed['FORMA_INGRESSO_TITULAR'].astype(bool)
df_transformed['STATUS_SERVENTIA'] = df_transformed['STATUS_SERVENTIA'].astype(bool)
df_transformed['ATRIBUICAO'] = df_transformed['ATRIBUICAO'].astype(str)

# pd.to_datetime(df_transformed['DT_INATIVACAO'], format='%Y-%m-%d', errors='coerce', )

df_transformed

In [ ]:
df_transformed.head().to_excel(path.joinpath('.xlsx'), index=False)

In [ ]:
df_transformed.to_parquet(path.joinpath('.parquet'), index=False)

In [ ]:
df_transformed.columns

In [ ]:
df_renamed = df_transformed.rename(columns={
    'SEQ':                           'seq',
    'CNS':                           'cns',
    'CPNJ':                          'cpnj',
    'DENOMINACAO_SERVENTIA':         'denominacao_serventia',
    'STATUS_SERVENTIA':              'status_serventia',
    'ATRIBUICAO':                    'atribuicao',
    'DT_INSTALACAO':                 'dt_instalacao',
    'DAT_INCLUSAO':                  'dat_inclusao',
    'DAT_ALTERACAO':                 'dat_alteracao',
    'UF':                            'uf',
    'MUNICIPIO':                     'municipio',
    'COD_IBGE':                      'cod_ibge',
    'BAIRRO':                        'bairro',
    'DISTRITO':                      'distrito',
    'SUB_DISTRITO':                  'sub_distrito',
    'ENDERECO':                      'endereco',
    'COMPLEMENTO':                   'complemento',
    'CEP':                           'cep',
    'TELEFONE1':                     'telefone_1',
    'FAX':                           'fax',
    'EMAIL':                         'email',
    'HOME_PAGE':                     'home_page',
    'DT_INATIVACAO':                 'dt_inativacao',
    'NOME_TITULAR':                  'nome_titular',
    'CPF_TITULAR':                   'cpf_titular',
    'DT_INGRESSO_TITULAR':           'dt_ingresso_titular',
    'DT_NOMEACAO_TITULAR':           'dt_nomeacao_titular',
    'TIPO_TITULAR':                  'tipo_titular',
    'FORMA_INGRESSO_TITULAR':        'forma_ingresso_titular',
    'DT_ASSUNCAO_SERVENTIA_TITULAR': 'dt_assuncao_serventia_titular',
    'DT_COLACAO_GRAU_TITULAR':       'dt_colacao_grau_titular',
    'BACHARELADO_TITULAR':           'bacharelado_titular',
    'NOME_SUBSTITUTO':               'nome_substituto',
    'CPF_SUBSTITUTO':                'cpf_substituto',
    'EMAIL_SUBSTITUTO':              'email_substituto',
    'DAT_INCLUSAO_SUBSTITUTO':       'dat_inclusao_substituto',
    'DAT_ALTERACAO_SUBSTITUTO':      'dat_alteracao_substituto',
})
df_renamed.to_parquet(path.joinpath('parquet'), index=False)


dev-onr-data-lake.cnj.correge_extra_judicial